# Review: Querying an API endpoint

### Mapbox Geocoding API

Services like Google Maps and Mapbox have various APIs that let you access its services through code instead of through GUI apps. This one from Mapbox lets you look up the latitude-longitude coordinates of street addresses.

It works similarly to the earthquakes example, but with query parameters added to the URL endpoint!

**API documentation:**  
https://www.mapbox.com/api-documentation/#geocoding

**API endpoint:**  
https://api.mapbox.com/geocoding/v5/mapbox.places

**API endpoint with query parameters:**  
https://api.mapbox.com/geocoding/v5/mapbox.places/Wurster+Hall.json?access_token=pk.eyJ1IjoiY3AyNTVkZW1vIiwiYSI6ImRPcTlnTUEifQ.3C0d0Nk_rcwV-8JF29PU-w


## Setting up API Keys for this session

You can get your own access key by signing up for a Mapbox account. Here is a link for that (Go ahead and do it now):

https://www.mapbox.com/signin/?route-to=%22/account/access-tokens%22

The Census Bureau has many APIs that can be used directly from Python - we will explore one later in class today.  You'll need to request an API key here: 

https://api.census.gov/data/key_signup.html

Go ahead and do this, and check your email when we come to that part of the session that needs this.

## Reviewing the Mapbox API endpoint for geocoding

In [1]:
import json      # library for working with JSON-formatted text strings
import requests  # library for accessing content from web URLs

import pprint    # library for cleanly printing Python data structures
pp = pprint.PrettyPrinter()

In [ ]:
# we have to encode the search query so that it can be passed as a URL, 
# with spaces and other special characters removed

endpoint = 'https://api.mapbox.com/geocoding/v5/mapbox.places/'

address = 'Wurster Hall'

params = {'limit': 1,
          'access_token': 'pk.eyJ1IjoiY3AyNTVkZW1vIiwiYSI6ImRPcTlnTUEifQ.3C0d0Nk_rcwV-8JF29PU-w'}

url = requests.Request('GET', endpoint+address+'.json', params=params).prepare().url
print(url)

In [ ]:
# download and parse the results

response = requests.get(url)
results = response.text
data = json.loads(results)

print(data)

In [ ]:
# print it more nicely

pp.pprint(data)

In [ ]:
# pull out the lat-lon coordinates

for r in data['features']:
    coords = r['geometry']['coordinates']
    print(coords)

## Using the Mapbox Python SDK for Geocoding

So far the discussion of APIs has been based on just accessing the API endpoints, which could be done with anything that can access a properly specified url.  Even just your browser.

Now we will look at APIs from a different perspective, one that is more Pythonic.  We will first need to install the Mapbox SDK.

In [ ]:
#!pip install mapbox

You will need to manage access tokens for Mapbox APIs (and for many others).  Read this for more information:

https://github.com/mapbox/mapbox-sdk-py/blob/master/docs/access_tokens.md

### Forward Geocoding

Forward geocoding is the one we have looked at so far using an ALI endpoint.  Let's look at it again using the Mapbox Python SDK.

In [2]:
from mapbox import Geocoder
import os

First - a brief digression -- these access tokens should not be publicly exposed, like on a web page or github, since others could get them and abuse them, and they are connected to you, so you might get blocked (blacklisted) by the API providers.  So -- the best way to handle this is to store the API Key or access token in an environment variable.  Since we are using datahub and each account is accessible only via Calnet Authentication, this is not such a big deal, but you should be aware of it and manage it differently on your own computer - by storing them in environment variables on your computer.  

Each operating system handled environment variables differently, and that is something you should learn on your own.  Below is a simple example of setting them temporarily.  Usually you would add them to a file that gets read when you start a command shell.  For example on my mac it is in .bash_profile.

In [67]:
os.environ['MAPBOX_ACCESS_TOKEN'] = "pk.eyJ1IjoiY3AyNTVkZW1vIiwiYSI6ImRPcTlnTUEifQ.3C0d0Nk_rcwV-8JF29PU-w"

In [68]:
os.environ['MAPBOX_ACCESS_TOKEN']

'pk.eyJ1IjoiY3AyNTVkZW1vIiwiYSI6ImRPcTlnTUEifQ.3C0d0Nk_rcwV-8JF29PU-w'

OK, now we can get to work on the geocoding... let's try geocoding Freehouse Restaurant, which is at 2700 Bancroft Way.

In [70]:
geocoder = Geocoder(access_token = os.environ['MAPBOX_ACCESS_TOKEN'])

In [72]:
response = geocoder.forward('2700 Bancroft Way, Berkeley, CA 94704', limit=1)

In [75]:
results = response.text

In [76]:
data = json.loads(results)
pp.pprint(data)

{'attribution': 'NOTICE: © 2018 Mapbox and its suppliers. All rights reserved. '
                'Use of this data is subject to the Mapbox Terms of Service '
                '(https://www.mapbox.com/about/maps/). This response and the '
                'information it contains may not be retained. POI(s) provided '
                'by Foursquare.',
 'features': [{'address': '2700',
               'center': [-122.25425, 37.868969],
               'context': [{'id': 'neighborhood.285002',
                            'text': 'Telegraph Avenue'},
                           {'id': 'postcode.15012008988668840',
                            'text': '94704'},
                           {'id': 'place.4062647275990170',
                            'text': 'Berkeley',
                            'wikidata': 'Q484678'},
                           {'id': 'region.3591',
                            'short_code': 'US-CA',
                            'text': 'California',
                            'w

Another random address to geocode:

In [17]:
response = geocoder.forward('120 East 13th Street, Manhattan, New York, New York 10003', limit = 1)
results = response.text
first = response.geojson()['features'][0]
print(first['place_name'])
print([coord for coord in first['geometry']['coordinates']])

120 East 13th Street, Manhattan, New York, New York 10003, United States
[-73.988893, 40.733003]


### Your turn

Try geocoding 3 addresses you know. 

Can you see a good way to write a loop to geocode those addresses and add them to a dataframe?

### Reverse Geocoding

Reverse geocoding does what it sounds like. It takes coordinates and returns an address or other place type.  Options for place type include: country, region, postcode, district, place, locality, neighborhood, address,and poi.

Let's start by using the coordinates of the address we just got from forward geocoding, and get the address back from reverse geocoding of those coordinates.

In [87]:
response = geocoder.reverse(lon=-73.988893, lat=40.733003)
features = response.geojson()['features']
features

[{'address': '120',
  'center': [-73.988893, 40.733003],
  'context': [{'id': 'neighborhood.2103290', 'text': 'Greenwich Village'},
   {'id': 'locality.12696928000137850',
    'text': 'Manhattan',
    'wikidata': 'Q11299'},
   {'id': 'postcode.13482670360296810', 'text': '10003'},
   {'id': 'place.15278078705964500', 'text': 'New York', 'wikidata': 'Q60'},
   {'id': 'region.3866',
    'short_code': 'US-NY',
    'text': 'New York',
    'wikidata': 'Q1384'},
   {'id': 'country.3145',
    'short_code': 'us',
    'text': 'United States',
    'wikidata': 'Q30'}],
  'geometry': {'coordinates': [-73.988893, 40.733003], 'type': 'Point'},
  'id': 'address.4216698541380788',
  'place_name': '120 East 13th Street, Manhattan, New York, New York 10003, United States',
  'place_type': ['address'],
  'properties': {},
  'relevance': 1,
  'text': 'East 13th Street',
  'type': 'Feature'},
 {'bbox': [-74.005282, 40.72586, -73.98734, 40.73907],
  'center': [-74.0029, 40.7284],
  'context': [{'id': 'local

In [88]:
for f in features:
    print('{place_name}'.format(**f))

120 East 13th Street, Manhattan, New York, New York 10003, United States
Greenwich Village, Manhattan, New York, New York 10003, United States
Manhattan, 10003, New York, New York, United States
New York, New York 10003, United States
New York, New York, United States
New York, United States
United States


Now let's look up Freehouse Restaurant's address from its Lat Long

In [39]:
response = geocoder.reverse(lon=-122.25425, lat=37.868969, types=['address'])  
features = response.geojson()['features']
features[0]['place_name']

'2700 Bancroft Way, Berkeley, California 94704, United States'

### Your turn

Try forward geocoding an address, and then reverse geocoding the coordinates to get the address back. Use your home address and see if it works.  Try a second address.

### Getting Directions from Mapbox

You can put in start and end locations and call the Mapbox directions API to get suggested directions as a sequence of coordinates you could plot on a map.

In [42]:
import mapbox
help(mapbox.Directions)

Help on class Directions in module mapbox.services.directions:

class Directions(mapbox.services.base.Service)
 |  Access to the Directions v5 API.
 |  
 |  Method resolution order:
 |      Directions
 |      mapbox.services.base.Service
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  directions(self, features, profile='mapbox/driving', alternatives=None, geometries=None, overview=None, steps=None, continue_straight=None, waypoint_snapping=None, annotations=None, language=None, **kwargs)
 |      Request directions for waypoints encoded as GeoJSON features.
 |      
 |      Parameters
 |      ----------
 |      features : iterable
 |          An collection of GeoJSON features
 |      profile : str
 |          Name of a Mapbox profile such as 'mapbox.driving'
 |      alternatives : bool
 |          Whether to try to return alternative routes, default: False
 |      geometries : string
 |          Type of geometry returned (geojson, polyline, polyline6)
 |      overview : 

In [43]:
from mapbox import Directions

In [44]:
service = Directions()

In [45]:
origin = {
        'type': 'Feature',
        'properties': {'name': 'Portland, OR'},
        'geometry': {
        'type': 'Point',
        'coordinates': [-122.7282, 45.5801]}}
destination = {
    'type': 'Feature',
    'properties': {'name': 'Bend, OR'},
    'geometry': {
    'type': 'Point',
    'coordinates': [-121.3153, 44.0582]}}

In [46]:
response = service.directions([origin, destination],'mapbox/driving')

In [47]:
driving_routes = response.geojson()

In [48]:
driving_routes

{'features': [{'geometry': {'coordinates': [(45.57994, -122.72832),
     (45.56995, -122.69555),
     (45.52473, -122.66403),
     (45.53161, -122.56801),
     (45.54728, -122.55336),
     (45.53943, -122.41837),
     (45.45508, -122.37623),
     (45.3759, -122.2218),
     (45.36648, -122.15453),
     (45.37977, -122.04793),
     (45.30527, -121.87119),
     (45.3123, -121.79323),
     (45.30066, -121.73499),
     (45.15902, -121.66228),
     (45.10711, -121.55876),
     (45.05603, -121.51364),
     (45.02217, -121.51322),
     (44.86717, -121.42421),
     (44.78583, -121.32094),
     (44.76025, -121.22736),
     (44.71869, -121.22773),
     (44.72038, -121.17616),
     (44.65345, -121.12858),
     (44.45997, -121.19954),
     (44.27523, -121.16913),
     (44.05817, -121.31533)],
    'type': 'LineString'},
   'properties': {'distance': 269397.3, 'duration': 11626},
   'type': 'Feature'}],
 'type': 'FeatureCollection'}

**Soon we will begin working with GeoJson files and mapping them... but not today.

### Your turn

Try generating directions from your home address to Freehouse restaurant.

### Using the Census API to reverse geocode to get FIPS codes (or other data)

Now check your email for the Census API - it should have been processed by now.  Copy that access key and we will use it below.

In [54]:
import requests
from requests.auth import HTTPBasicAuth

apikey = "c4892baf28a2c18b2d3cd9ce9f68ebacecd2a970"
request_url = "http://citysdk.commerce.gov"

request_obj = {
  'zip': '21401',
  'state': 'MD',
  'level': 'state',
  'sublevel': False,
  'api': 'acs5',
  'year': 2010,
  'variables': ['income', 'population']
}

response = requests.post(request_url, auth=HTTPBasicAuth(apikey, None), json=request_obj)



In [56]:
response

<Response [503]>

## Part 4: Reverse geocoding to FIPS

We'll use the FCC's Census Block Conversions API to turn lat/long into a block FIPS code. FIPS codes contain from left to right: the location's 2-digit state code, 3-digit county code, 6-digit census tract code, and 4-digit census block code (the first digit of which is the census block group code). Now you can join your data to tract (etc) level census data without doing a spatial join.

- Documentation: https://geo.fcc.gov/api/census/

In [65]:
url = 'https://geo.fcc.gov/api/census/block/find?latitude=34.537094&longitude=-82.630303&format=json'
response = requests.get(url)
data = response.json()
data

{'Block': {'FIPS': '450070003002024',
  'bbox': [-82.636071, 34.535797, -82.630107, 34.540163]},
 'County': {'FIPS': '45007', 'name': 'Anderson'},
 'State': {'FIPS': '45', 'code': 'SC', 'name': 'South Carolina'},
 'executionTime': '0',
 'messages': ["FCC0001: The coordinate lies on the boundary of mulitple blocks, the block contains the clicked location is selected. For a complete list use showall=true to display 'intersection' element in the Block"],
 'status': 'OK'}

In [66]:
data['Block']['FIPS']

'450070003002024'

### Your turn: 

1. Look up the latitude and longitude of your home location using forward geocoding
2. Use the latitude and longitude from that to get the census block of your home location
3. If you have time, create a list of 3 street addresses, and write code that will iterate through them, looking up the latitude and longitude, and the FIPS block code, and write these addresses, lat and long, and FIPS Block code to a dataframe.